Since the HW grading is done in a semi-automatic manner, please adhere to the following naming format for your submission.
Each group of students (mostly pairs, with some approved exceptions) should submit a Jupyter notebook (.ipynb file and not a .zip file) whose name is the underscored-separated id list of all the submitters. 
For example, for two submitters, the naming format is: id1_id2.ipynb.

# Question 1

a) Download the "Boston1.csv" database, and explore the data. Explanation about the dataset can be found here: http://www.clemson.edu/economics/faculty/wilson/R-tutorial/analyzing_data.html

Find the columns with missing values and filter them out of the data.

In [54]:
import pandas as pd

df = pd.read_csv('Boston1.csv')
cols_with_missing = [col for col in df.columns if df[col].isnull().any()]
print('Columns with missing values: ', cols_with_missing)

# Filter out the columns with missing values
df_filtered = df.drop(columns=cols_with_missing)

Columns with missing values:  ['misData']


b) Divide the filtered data randomly into a train set (70% of the data) and test set (30% of the data).

In [55]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df_filtered, test_size=0.3, random_state=42)

# Question 2

If you haven't done this previously, install the scikit-learn package for python.

a) On the train set, run a linear regression model as follows:
Divide the training set into explanatory variables (the X matrix with which we'll try to make a prediction) and a target variable (y, the value which we'll try to predict). Use the 'medv' attribute as the target variable y and the rest of the features as the X matrix. Run a linear regression model on those sets, and print the regression coefficients. 

In [56]:
from sklearn.linear_model import LinearRegression

mdl     = LinearRegression()
train_x = train_df.drop(['medv'],axis=1)
train_y = train_df['medv']
test_x  = test_df.drop(['medv'],axis=1)
test_y  = test_df['medv']

mdl.fit(train_x, train_y)
m = mdl.coef_
print(m)

[-1.31554143e-01  3.56062850e-02  4.76542127e-02  3.14666736e+00
 -1.51275836e+01  4.07563132e+00 -1.10715557e-02 -1.38369791e+00
  2.41853708e-01 -8.74151558e-03 -9.10127712e-01  1.18064380e-02
 -5.46257067e-01 -5.51868467e-01]


b) Use the linear regression model to predict the values of the test set's 'medv' column, based on the test set's other attributes. Print the Mean Squared Error of the model on the train set and on the test set.
Usually, the MSE on the train set would be lower than the MSE on the test set, since the model parameters are optimized with respect to the train set. Must this always be the case? Can you think of a few examples for when this might not be the case?

In [57]:
from sklearn.metrics import mean_squared_error as mse

print("MSE of the train set:\n", mse(mdl.predict(train_x), train_y))
print("MSE of the test set:\n",mse(mdl.predict(test_x), test_y))
print("1) Noise in the Training Data: If the training data has a significant amount of noise or outliers that the model tries to fit, the training MSE might be high. On the other hand, if the test data is cleaner without such noise or outliers, the test MSE might be lower.\n2) Regularization: If a model is trained with strong regularization, it may be underfit to the training set and thus have a higher training error.\n3) Small Test Set: If the test set is relatively small, it might just by chance contain easier examples that the model can predict more accurately, resulting in a lower test MSE.")

MSE of the train set:
 22.52265349764171
MSE of the test set:
 21.774657824501453
1) Noise in the Training Data: If the training data has a significant amount of noise or outliers that the model tries to fit, the training MSE might be high. On the other hand, if the test data is cleaner without such noise or outliers, the test MSE might be lower.
2) Regularization: If a model is trained with strong regularization, it may be underfit to the training set and thus have a higher training error.
3) Small Test Set: If the test set is relatively small, it might just by chance contain easier examples that the model can predict more accurately, resulting in a lower test MSE.


c) Add some noise (with mean=0, std=1) to the test set's y, and predict it again. What happened to the MSE? Why?

In [58]:
import numpy as np
# Add some noise
noise = np.random.normal(0, 1, len(test_y.index))
noised_test_y = test_y + noise
print("MSE of the test set after adding some noise:\n",mse(mdl.predict(test_x), noised_test_y))

MSE of the test set after adding some noise:
 21.85764233190693


# Question 3

a) Create a Recursive feature elimination model, with a linear regression estimator, that selects half of the original number of features. Hint: Check the feature_selection module in scikit-learn.

In [59]:
from sklearn.feature_selection import RFE

# Thanks for your hint
rfe = RFE(mdl)

b) Use the feature elimination model on the full database (after filtering columns with missing values, before partitioning into train/test). Print the features that were selected. Remember that we separate the 'medv' attribute to be our y, while the rest of the attributes in the dataset serve as features to learn from.

In [60]:
X = df_filtered.drop(['medv'],axis=1)
y = df_filtered['medv']
selected_model = rfe.fit(X,y)
print("The features that were selected:")
print(X.columns[selected_model.support_].values)

The features that were selected:
['chas' 'nox' 'rm' 'dis' 'ptratio' 'lstat' 'randCol']


c) We'd like to find out the optimal number of features. Create feature elimination models (with linear regression estimators) for every number of features between 1 and n (where n = all the original features, 'medv' excluded). For each number of features, run a linear regression as in Question 2, only on the selected features, in order to predict 'medv'. Print the Mean Sqaured Error for each number of features.

In [63]:
n=X.shape[1]
mses = list()
for i in range (1,n+1):
    rfe_i = RFE(estimator=mdl, n_features_to_select=i)
    selected_model = rfe_i.fit(X,y)
    selected_features = X.columns[selected_model.support_].values
    selected_X = X.reindex(columns = selected_features)
    mdl.fit(selected_X,y)
    mse_i = mse(y, mdl.predict(selected_X))
    print("For",i,"selected features the MSE is:",mse_i)
    mses.append(mse_i)

For 1 selected features the MSE is: 69.0042883554067
For 2 selected features the MSE is: 39.21811674276104
For 3 selected features the MSE is: 37.518269887684575
For 4 selected features the MSE is: 32.446947181499546
For 5 selected features the MSE is: 30.929546079491566
For 6 selected features the MSE is: 23.99421489307857
For 7 selected features the MSE is: 23.988679676026152
For 8 selected features the MSE is: 23.873140175553736
For 9 selected features the MSE is: 23.34492967422981
For 10 selected features the MSE is: 23.255560445416666
For 11 selected features the MSE is: 22.930361610394424
For 12 selected features the MSE is: 22.42502862441528
For 13 selected features the MSE is: 21.880860827853045
For 14 selected features the MSE is: 21.880721616729907


d) Conclude the optimal number of features for this task. Think about the cost of adding for data vs the benefit of a more accurate prediction. Explain your answer.

In [ ]:
print("From the results, we can point for 2 massive MSE drops.\nOne from moving from 1 to 2 features, and another from moving from 5 to 6 features.")
print("In this case, we would conclude that the optimal number of features for this task is 6. Due to the cost of adding for data vs the benefit of a more accurate prediction. Because we could pick all the features (as seen have the lowest MSE) but the cost would be huge.")

# Question 4

Perform a cross-validation of the linear regression on the train set with K=5. Print the CV scores for each repeat.

In [64]:
from sklearn.model_selection import cross_val_score

print(cross_val_score(mdl, train_x, train_y, cv=5))

[0.74512757 0.51729534 0.75283635 0.76628987 0.6464534 ]
